# Analysis based on Effective number of candidates 

## Lok Sabha

In [3]:
import numpy as np
# following data is copy pasted from eci website
l = ["Bharatiya Janata Party - BJP	239	1	240" , "Indian National Congress - INC	96	3	99" , "Samajwadi Party - SP	37	0	37" , "All India Trinamool Congress - AITC	29	0	29" , "Dravida Munnetra Kazhagam - DMK	20	2	22" , "Telugu Desam - TDP	16	0	16" , "Janata Dal (United) - JD(U)	12	0	12" , "Shiv Sena (Uddhav Balasaheb Thackrey) - SHSUBT	9	0	9" , "Nationalist Congress Party Sharadchandra Pawar - NCPSP	7	1	8" , "Shiv Sena - SHS	7	0	7" , "Lok Janshakti Party(Ram Vilas) - LJPRV	5	0	5" , "Yuvajana Sramika Rythu Congress Party - YSRCP	4	0	4" , "Rashtriya Janata Dal - RJD	4	0	4" , "Communist Party of India (Marxist) - CPI(M)	4	0	4" , "Indian Union Muslim League - IUML	3	0	3" , "Aam Aadmi Party - AAAP	3	0	3" , "Jharkhand Mukti Morcha - JMM	3	0	3" , "Janasena Party - JnP	2	0	2" , "Communist Party of India (Marxist-Leninist) (Liberation) - CPI(ML)(L)	2	0	2" , "Janata Dal (Secular) - JD(S)	2	0	2" , "Viduthalai Chiruthaigal Katchi - VCK	2	0	2" , "Communist Party of India - CPI	2	0	2" , "Rashtriya Lok Dal - RLD	2	0	2" , "Jammu & Kashmir National Conference - JKN	2	0	2" , "United People’s Party Liberal - UPPL	1	0	1" , "Asom Gana Parishad - AGP	1	0	1" , "Hindustani Awam Morcha (Secular) - HAMS	1	0	1" , "Kerala Congress - KEC	1	0	1" , "Revolutionary Socialist Party - RSP	1	0	1" , "Nationalist Congress Party - NCP	1	0	1" , "Voice of the People Party - VOTPP	1	0	1" , "Zoram People’s Movement - ZPM	1	0	1" , "Shiromani Akali Dal - SAD	1	0	1" , "Rashtriya Loktantrik Party - RLTP	1	0	1" , "Bharat Adivasi Party - BHRTADVSIP	1	0	1" , "Sikkim Krantikari Morcha - SKM	1	0	1" , "Marumalarchi Dravida Munnetra Kazhagam - MDMK	1	0	1" , "Aazad Samaj Party (Kanshi Ram) - ASPKR	1	0	1" , "Apna Dal (Soneylal) - ADAL	1	0	1" , "AJSU Party - AJSUP	1	0	1" , "All India Majlis-E-Ittehadul Muslimeen - AIMIM	1	0	1" , "Independent - IND	7	0	7"]
l = [x.split()[-3:] for x in l]
l = [[int(x) for x in y] for y in l]
total_seats = np.sum([x[0] for x in l])
sum = 0
sum = 1/np.sum([ (x[0]/total_seats)**2 for x in l])
sum = round(sum, 2)
print ("Enp for 2024 Lok Sabha Elections (WITHOUT consider coalitions) is ", sum)

l = ["NDA 293", "INDIA 232", "Others 18"]
l = [x.split()[-1:] for x in l]
l = [[int(x) for x in y] for y in l]
total_seats = np.sum([x[0] for x in l])
sum = 0
sum = 1/np.sum([ (x[0]/total_seats)**2 for x in l])
sum = round(sum, 2)
print ("Enp for 2024 Lok Sabha Elections (consider coalitions) is ", sum)



Enp for 2024 Lok Sabha Elections (WITHOUT consider coalitions) is  4.12
Enp for 2024 Lok Sabha Elections (consider coalitions) is  2.11


## State Assembly

In [4]:
import numpy as np
import pandas as pd
data = pd.read_csv('data/results_2024_winners.csv')
data = np.array(data)
data = data[:,[0,4]]
d = {}
for x in data:
    if x[0] not in d:
        d[x[0]] = {}
    if x[1] not in d[x[0]]:
        d[x[0]][x[1]] = 0
    d[x[0]][x[1]] += 1
# list of parties
parties = []
for state in d:
    for party in d[state]:
        if party not in parties:
            parties.append(party)
d['Parties'] = {}
for party in parties:
    d['Parties'][party] = 0
df = pd.DataFrame(d)
df = df.T
df.index.name = 'State'
df_data = df

# calculate enp for each state
enp = {}
for state in d:
    total_seats = np.sum([d[state][x] for x in d[state]])
    sum = 1/np.sum([ (d[state][x]/total_seats)**2 for x in d[state]])
    sum = round(sum, 2)
    enp[state] = sum
df = pd.DataFrame(enp.items(), columns=['State', 'Enp'])
df_data = pd.merge(df, df_data, on='State')
df_data.to_csv('output/results_2024_states.csv', index= False)


/tmp/ipykernel_6571/3550746592.py:31: RuntimeWarning: invalid value encountered in scalar divide
  sum = 1/np.sum([ (d[state][x]/total_seats)**2 for x in d[state]])


## District 

In [5]:
import numpy as np
import pandas as pd
data = pd.read_csv('data/results_2024.csv')
data = np.array(data)
new_data = {}
for i in np.unique(data[:,0]):
    i_data = data[data[:,0] == i]
    arr = {}
    for j in np.unique(i_data[:,2]):
        # make a list of all the rows with the same state and party
        l = data[(data[:,0] == i) & (data[:,2] == j)][:,9]
        arr[j] = l
    new_data[i] = arr
for i in new_data:
    sum_i = 0
    for j in new_data[i]:
        list_percent = new_data[i][j]
        for k in range(len(list_percent)):
            id=0
            try :
                id = float(list_percent[k])
            except:
                id = 0
            list_percent[k] = id
        if np.sum(list_percent) <99.9:
            continue
        enp = 1/np.sum([ (x/100)**2 for x in list_percent])
        new_data[i][j] = [enp, list_percent]
    # average enp
    sum_i = np.sum([new_data[i][j][0] for j in new_data[i]])/len(new_data[i])
    new_data[i] = [sum_i, new_data[i]]
lines = ["State,Party,ENP,Avg ENP\n"]
for i in new_data:
    for j in new_data[i][1]:
        # 2 decimal points
        lines.append(f"{i},{j},{new_data[i][1][j][0]:.2f},{new_data[i][0]:.2f}\n")
with open('output/results_2024_district.csv', 'w') as f:
    f.writelines(lines)



In [6]:
cat = [1.8, 2.4, 3]
count = [0, 0, 0, 0]
for i in new_data:
    for j in new_data[i][1]:
        enp = new_data[i][1][j][0]
        if enp < cat[0]:
            count[0] += 1
        elif enp < cat[1]:
            count[1] += 1
        elif enp < cat[2]:
            count[2] += 1
        else:
            count[3] += 1
num = [[["ENP < 1.8",f"{count[0]}"]], [["ENP 1.8-2.4",f"{count[1]}"]], [["ENP 2.4-3",f"{count[2]}"]], [["ENP > 3",f"{count[3]}"]]]
for i in new_data:
    for j in new_data[i][1]:
        enp = new_data[i][1][j][0]
        if enp < cat[0]:
            num[0].append([i, j])
        elif enp < cat[1]:
            num[1].append([i, j])
        elif enp < cat[2]:
            num[2].append([i, j])
        else:
            num[3].append([i, j])
num = [num[1], num[2], num[3], num[0]]
lines = []
for i in range(len(num[0])):
    lines.append(f"{num[0][i][0]},{num[0][i][1]}")
for i in range(len(num[1])):
    lines[i] += f",{num[1][i][0]},{num[1][i][1]}"
for i in range(len(num[2])):
    lines[i] += f",{num[2][i][0]},{num[2][i][1]}"
for i in range(len(num[3])):
    lines[i] += f",{num[3][i][0]},{num[3][i][1]}"
for i in range(len(lines)):
    lines[i] += "\n"
with open('output/results_2024_district_cat.csv', 'w') as f:
    f.writelines(lines)

